In [1]:
import sys
import json
import os
import string
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader

if r'G:\PythonProjects\WineRecognition2' not in sys.path:
    sys.path.insert(0, r'G:\PythonProjects\WineRecognition2')

from nn.utils import generate_tag_to_ix, get_model_confidence, get_model_mean_confidence, CustomDataset
from nn.mlflow_utils import log_mlflow_on_test
from nn.model import BiLSTM_CRF
from data_master import DataGenerator, count_unk_foreach_tag, DataAnalyzer
from tqdm.notebook import tqdm

import neptune
from neptune.types import File

from sklearn_crfsuite import metrics

D:\ProgramFiles\anaconda3\envs\wine-recognition\lib\site-packages\papermill\iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [2]:
run = neptune.init_run(
    project="mentalakv/wine-recognition",
    api_token=os.getenv('NEPTUNE_API_KEY'),
    capture_stderr=True,
    capture_stdout=True,
    capture_traceback=True,
    capture_hardware_metrics=True,
    dependencies='infer'
)

https://app.neptune.ai/mentalakv/wine-recognition/e/WIN-21


In [3]:
TRAIN_RUN_ID = 'WIN-17'
RUN_NAME = 'Test-100-256'
START_TIME = ''
OUTPUT_DIR = 'G:/PythonProjects/WineRecognition2/artifacts/test/test'
DATA_PATH = r'G:\PythonProjects\WineRecognition2\data\text\menu_txt_tagged_fixed_bottlesize.txt'
COMPUTE_METRICS = True
DEVICE = 'cpu'

In [5]:
train_run = neptune.init_run(with_id=TRAIN_RUN_ID, mode="read-only", api_token=os.getenv('NEPTUNE_API_KEY'), project="mentalakv/wine-recognition")
train_run['model_checkpoints/best_model'].download(destination=OUTPUT_DIR)
train_run['data/vocab'].download(destination=OUTPUT_DIR)
train_run['data/tags'].download(destination=OUTPUT_DIR)
train_run_params = train_run['parameters'].fetch()

https://app.neptune.ai/mentalakv/wine-recognition/e/WIN-17


In [ ]:
params = {
    'train_run_id': TRAIN_RUN_ID,
    'data_path': DATA_PATH,
    'compute_metrics': COMPUTE_METRICS,
    'device': DEVICE
}

run['run_info'] = {
    'name': RUN_NAME,
    'output_directory': OUTPUT_DIR,
    'start_time': START_TIME
}

run['train_run_parameters'] = train_run_params
run['parameters'] = params
run['sys/tags'].add([train_run_params['model_name'], 'test'])

In [6]:
with open(os.path.join(OUTPUT_DIR, 'vocab.json'), 'r', encoding='utf-8') as file:
    word_to_ix = json.load(file)

vocab_size = len(word_to_ix) 
    
vocab_size

18208

In [19]:
with open(os.path.join(OUTPUT_DIR, 'tags.json')) as file:
    tag_to_ix = json.load(file)
    
num_tags = len(tag_to_ix)

if not COMPUTE_METRICS:
    tag_to_ix['UNKNOWN'] = max(tag_to_ix.values()) + 1

ix_to_tag = {value: key for key, value in tag_to_ix.items()}
tag_to_ix

{'Add_TradeName': 0,
 'Add_Brand': 1,
 'Add_KeyWordTrue': 2,
 'Add_KeyWordFalse': 3,
 'Add_GrapeVarieties': 4,
 'Add_GeoIndication': 5,
 'Add_WineType': 6,
 'Add_BottleSize': 7,
 'Add_Sweetness': 8,
 'Add_WineColor': 9,
 'Add_ClosureType': 10,
 'Add_Certificate': 11,
 'Add_Vintage': 12,
 'Add_Price': 13,
 'Punctuation': 14,
 'Other': 15}

In [8]:
with open(DATA_PATH, encoding='utf-8') as file:
    x_test = DataGenerator.generate_sents2(file.read().split('\n'))

    
dataset = CustomDataset(
    x_test,
    tag_to_ix,
    word_to_ix,
    case_sensitive=train_run_params['case_sensitive_vocab'],
    convert_nums2words=train_run_params['use_num2words']
)

y_test = [tags for _, tags in dataset.raw_data()]
dataloader = DataLoader(dataset, batch_size=2048, shuffle=False, drop_last=False)

In [9]:
model = BiLSTM_CRF(vocab_size, num_tags, train_run_params['embedding_dim'], train_run_params['hidden_dim'], word_to_ix['PAD'])
model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, 'best_model.pth')))
model = model.to(DEVICE)
model.eval()

BiLSTM_CRF(
  (embedding): Embedding(18208, 64, padding_idx=18173)
  (lstm): LSTM(64, 64, batch_first=True, bidirectional=True)
  (hidden2tags): Linear(in_features=128, out_features=16, bias=True)
  (crf): CRF(num_tags=16)
)

In [24]:
y_pred = []
with torch.no_grad():
    for x, tags, mask, _ in dataloader:
        x = x.to(DEVICE)
        tags = tags.to(DEVICE)
        mask = mask.to(DEVICE)
        best_tag_seq = model(x, mask)
        y_pred.extend(best_tag_seq)

In [11]:
x_tensor = [
    torch.tensor(dataset.sentence_to_indices(sentence), dtype=torch.int64) for sentence, _ in dataset.raw_data()
]

In [ ]:
unk_foreach_tag = count_unk_foreach_tag(x_tensor, y_test, list(tag_to_ix), dataset.word_to_ix[dataset.unk])

unk_foreach_tag_path = os.path.join(OUTPUT_DIR, 'unk_foreach_tag.json')
with open(unk_foreach_tag_path, 'w', encoding='utf-8') as file:
    json.dump(unk_foreach_tag, file)
    
run['results/unk_foreach_tag'].upload(unk_foreach_tag_path)

In [32]:
conf = get_model_mean_confidence(model, x_tensor, DEVICE, tqdm)
run['metrics/confidence'] = conf

0it [00:00, ?it/s]

In [25]:
for index, pred in enumerate(y_pred):
    y_pred[index] = [ix_to_tag[tag] for tag in pred]

x_test = [sentence for sentence, _ in dataset.raw_data()]

In [26]:
labels=list(tag_to_ix)

run['metrics/f1'] = metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)
run['metrics/precision'] = metrics.flat_precision_score(y_test, y_pred, average='weighted', labels=labels)
run['metrics/recall'] = metrics.flat_recall_score(y_test, y_pred, average='weighted', labels=labels)
run['metrics/accuracy'] = metrics.flat_accuracy_score(y_test, y_pred)

flat_class_report_path = os.path.join(OUTPUT_DIR, 'flat-classification-report.txt')
with open(flat_class_report_path, 'w', encoding='utf-8') as file:
    file.write(metrics.flat_classification_report(y_test, y_pred, labels=labels, digits=3))
    
run['metrics/flat_classification_report'].upload(flat_class_report_path)

D:\ProgramFiles\anaconda3\envs\wine-recognition\lib\site-packages\sklearn\metrics\_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
D:\ProgramFiles\anaconda3\envs\wine-recognition\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramFiles\anaconda3\envs\wine-recognition\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramFiles\anaconda3\envs\wine-recognition\lib\site-packages\sk

In [27]:
results_path = os.path.join(OUTPUT_DIR, 'results.txt')
with open(results_path, 'w', encoding='utf-8') as file:
    for sentence, tags in zip(x_test, y_pred):
        f = ['%-20s'] * len(sentence)
        file.write(' '.join(f) % tuple(tags) + '\n')
        file.write(' '.join(f) % tuple(sentence) + '\n')

run['results/model_output'].upload(results_path)

In [36]:
if COMPUTE_METRICS:
    test_eval = []
    for sentence, true_tags, pred_tags in zip(x_test, y_test, y_pred):
        test_eval.append(list(zip(sentence, true_tags, pred_tags)))
        
    colored_table_path = os.path.join(OUTPUT_DIR, 'colored-table.xlsx')
    diagram_path = os.path.join(OUTPUT_DIR, 'diagram.png')

    DataAnalyzer.analyze(
            test_eval,
            keys=labels,
            table_save_path=colored_table_path,
            diagram_save_path=diagram_path)
    
    run['results/colored_table'].upload(colored_table_path)
    run['results/diagram'].upload(diagram_path)

In [ ]:
df = []
classes = list(ix_to_tag.values())
for sentence, tags in zip(x_test, y_pred):
    output = [' '.join(word for word, tag in zip(sentence, tags) if tag == cls) for cls in classes]
    df.append(output)

source_strings = [' '.join(sentence) for sentence in x_test]
results_xlsx_path = os.path.join(OUTPUT_DIR, 'results.xlsx')
with pd.ExcelWriter(results_xlsx_path, engine='xlsxwriter') as writer:
    pd.DataFrame(df, columns=classes).to_excel(writer, sheet_name='results')
    pd.DataFrame(source_strings).to_excel(writer, sheet_name='source')

run['results/result_xlsx_table'].upload(results_xlsx_path)

In [37]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 5 operations to synchronize with Neptune. Do not kill this process.
All 5 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/mentalakv/wine-recognition/e/WIN-21/metadata
